<a href="https://colab.research.google.com/github/KAILASHVenkat/T3-PREPROCESSING/blob/main/preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers pandas datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00


In [2]:
import pandas as pd
from transformers import pipeline
from datasets import load_dataset

In [6]:
emotion_classifier = pipeline('text-classification', model='bhadresh-savani/bert-base-go-emotion')

def get_emotions(text):
    result = emotion_classifier(text)
    dominant_emotion = max(result, key=lambda x: x['score'])

    # Check if the score is greater than 0.5
    if dominant_emotion['score'] > 0.5:
        return dominant_emotion['label']
    else:
        return 'not_classified'

config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/333 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [3]:
# Load the MRPC dataset from Hugging Face
mrpc_dataset = load_dataset('glue', 'mrpc')
mrpc_df = pd.DataFrame({'input_text': mrpc_dataset['train']['sentence1'], 'target_text': mrpc_dataset['train']['sentence2']})


Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

In [4]:
mrpc_df.to_csv('output_mrpc_dataset.csv', index=False)

In [7]:
# Apply the get_emotions function to filter out emotions with score less than 0.5
mrpc_df['input_emotion'] = mrpc_df['input_text'].apply(get_emotions)
mrpc_df['target_emotion'] = mrpc_df['target_text'].apply(get_emotions)

# Filter out sentences with score less than 0.5
mrpc_df = mrpc_df[mrpc_df['input_emotion'] != 'not_classified']
mrpc_df = mrpc_df[mrpc_df['target_emotion'] != 'not_classified']

decreasing_intensity_pairs = mrpc_df[mrpc_df.apply(
    lambda row: emotion_classifier(row['input_text'])[0]['score'] > emotion_classifier(row['target_text'])[0]['score'],
    axis=1)]

increasing_intensity_pairs = mrpc_df.drop(decreasing_intensity_pairs.index)
increasing_intensity_pairs[['input_text', 'target_text']] = increasing_intensity_pairs[
    ['target_text', 'input_text']].reset_index(drop=True)

decreasing_intensity_pairs['input_text'] = decreasing_intensity_pairs.apply(
    lambda row: f"{row['input_emotion']} to {row['target_emotion']}: {row['input_text']}", axis=1)

final_mrpc_dataset = pd.concat([decreasing_intensity_pairs, increasing_intensity_pairs], ignore_index=True)

final_mrpc_dataset.to_csv('output_mrpc_dataset.csv', index=False)

<ipython-input-7-c67734ea117b>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  decreasing_intensity_pairs['input_text'] = decreasing_intensity_pairs.apply(


In [8]:
mrpc_df1 = pd.read_csv('output_mrpc_dataset.csv')
mrpc_df1.head()

,input_text,target_text,input_emotion,target_emotion
0,neutral to neutral: Amrozi accused his brother...,"Referring to him as only "" the witness "" , Amr...",neutral,neutral
1,neutral to neutral: They had published an adve...,"On June 10 , the ship 's owners had published ...",neutral,neutral
2,"neutral to neutral: Around 0335 GMT , Tab shar...","Tab shares jumped 20 cents , or 4.6 % , to set...",neutral,neutral
3,neutral to neutral: The Nasdaq had a weekly ga...,The tech-laced Nasdaq Composite .IXIC rallied ...,neutral,neutral
4,neutral to neutral: The DVD-CCA then appealed ...,The DVD CCA appealed that decision to the U.S....,neutral,neutral


In [9]:
# Save the DataFrame to a CSV file in Colab
mrpc_df1.to_csv('/content/1.csv', index=False)

In [10]:
from google.colab import files
files.download('/content/1.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>